**SFT (Supervised Fine-Tuning, instruction-tuning)**  — метод адаптации предварительно обученных языковых моделей (LLM) под конкретную задачу с помощью размеченных данных

**Цель** — скорректировать веса модели так, чтобы она лучше справлялась с задачей, не теряя при этом общие знания, полученные в ходе предварительного обучения

### Установка зависимостей

In [1]:
# !pip install --upgrade pip
# !pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# !pip install unsloth_zoo
# !pip install transformers accelerate
# !huggingface-cli login

# !pip install --upgrade pip
# !pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --force-reinstall unsloth_zoo

# !pip install --upgrade pip
# !pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --force-reinstall unsloth_zoo

### Model Gemma3-270m

In [ ]:
import unsloth
from unsloth import FastLanguageModel
from transformers import TextStreamer


MODEL_NAME = "google/gemma-3-270m"
MAX_SEQ = 2048

# загрузка модели (4bit, чтобы влезло в Colab T4)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ,
    load_in_4bit = True,
)

# оптимизация для инференса
FastLanguageModel.for_inference(model)

# простой запрос
prompt = "what is select * doing in SQL"

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(**inputs, streamer=streamer, max_new_tokens=200)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.4.1+cu121 with CUDA 1201 (you have 2.8.0+cu128)
    Python  3.12.6 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.4.1+cu121 with CUDA 1201 (you have 2.8.0+cu128)
    Python  3.12.6 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Gemma3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/

model.safetensors:   0%|          | 0.00/393M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

 to create a table on query

What is the main way to create a table on query?

What is the purpose of a table ?

what is the purpose of a table when a result is an ordered list of data ?

What is a table used in a data driven analysis?

What are the main purposes of a table ?

What is the main function of a table ?

What is the purpose of a query in a data driven analysis ?

what is the table used in a data driven analysis ?

What is a table used in a data driven analysis ?

What is meant by table?

what is a table used in a data driven analysis ?

what is a table used in a data driven analysis ?

what is the purpose of the following table

What is the main purpose of a table ?

What is the purpose of a table ?<eos>


### Model SQLCODER-7b

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
import unsloth
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

MODEL_NAME = "defog/sqlcoder-7b"

# загрузка модели
model_sql, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True
)


ModuleNotFoundError: No module named 'unsloth'

### Fine Tuning

#### Dataset

In [16]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="\data_processing\data\queries_marked_602.jsonl")

full_dataset = dataset["train"]

temp_split = full_dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)
train_dataset = temp_split["train"]
temp = temp_split["test"]

val_test = temp.train_test_split(test_size=0.5, seed=42, shuffle=True)

eval_dataset = val_test["train"]
test_dataset = val_test["test"]
print(train_dataset[0])
print(len(train_dataset), len(eval_dataset), len(test_dataset))

FileNotFoundError: Unable to find '\data_processing\data\queries_marked_602.jsonl'

#### Lora adaptor

In [ ]:
model = FastLanguageModel.get_peft_model(
    model_sql,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#### Setting up training parameters

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Параметры обучения
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    gradient_checkpointing=True,
    logging_steps=10,
    eval_steps=25,
    save_steps=25,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    greater_is_better=False,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    seed=3407,
    output_dir="fine-tuning/training_args",
    logging_dir="fine-tuning/training_args/logs",
    report_to="none",
)

#### Creating and launching training

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    prompt = example["instruction"]
    if example["input"].strip() != "":
        prompt += "\nInput: " + example["input"]
    prompt += "\nOutput: " + example["output"]
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_eval = eval_dataset.map(tokenize_function, batched=False)


tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    max_seq_length=512,
    packing=False,
    args=training_args,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 483 | Num Epochs = 3 | Total steps = 183
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
25,0.090700,0.560774
50,0.019000,0.856468
75,0.008700,1.035855
100,0.005300,1.033072
125,0.002400,1.161314
150,0.002200,1.338406
175,0.000500,1.250201


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=183, training_loss=0.29048734570629486, metrics={'train_runtime': 2272.6601, 'train_samples_per_second': 0.638, 'train_steps_per_second': 0.081, 'total_flos': 3.183858261216461e+16, 'train_loss': 0.29048734570629486, 'epoch': 3.0})

In [ ]:
model.save_pretrained("fine-tuning/Model/sqlcoder-finetuned")
tokenizer.save_pretrained("fine-tuning/Model/sqlcoder-finetuned")